# Part-2: MLP

Here we implement a multilayer perceptron (MLP) character-level language model using [Bengio et al. 2003 MLP language model paper.](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

### 1. Setting Up Environment and Loading the Dataset

In [2]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()# reading the file and splitting into list of
words[:8]# printing first 8 words

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)# number of words in the dataset

32033

In [5]:
#building the vocabulary of characters
chars = sorted(list(set(''.join(words))))# getting unique characters in the dataset and sorting them
stoi = {s:i+1 for i,s in enumerate(chars)}# mapping characters to integers (1-26)
stoi['.'] = 0# mapping the special character '.' to 0
itos = {i:s for s,i in stoi.items()}# mapping integers to characters
vocab_size = len(itos)# getting the vocabulary size
print('vocab size: ', vocab_size)
print(itos)

vocab size:  27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
